# Environment

In [1]:
%load_ext blackcellmagic

In [2]:
import pandas as pd
import seaborn as sns
import numpy as np

## Variables

In [3]:
TODAY = pd.Timestamp('2022-12-18').strftime('%Y-%m-%d')

In [4]:
PATH = ('/Users/kiralenz/Documents/keeprising/data/')

# Notes

In [5]:
date = pd.to_datetime('2023-03-25')

In [6]:
list_dates = []

In [7]:
list_dates.append(date)

In [8]:
list_dates

[Timestamp('2023-03-25 00:00:00')]

In [9]:
list_notes = []

In [10]:
notes = str('25g + 40g starter + 40 g flour reduces rising about 50%')

In [11]:
list_notes.append(notes)

In [12]:
list_notes

['25g + 40g starter + 40 g flour reduces rising about 50%']

In [13]:
notes_history = pd.DataFrame({
    'date':list_dates,
    'notes':list_notes
}, index=np.arange(0,len(list_dates)))

In [14]:
notes_history

,date,notes
0,2023-03-25,25g + 40g starter + 40 g flour reduces rising ...


In [15]:
notes_history.to_parquet(PATH + 'notes_history.parquet')

# Sourdough feeding data
creating a more or less true history

## Feeding dates

In [16]:
past_feeding_dates = pd.date_range(start='2022-07-24', end='2022-12-11', freq='7D')

In [17]:
feeding_dates = []

In [18]:
for i in past_feeding_dates:
  feeding_dates.append(i.strftime('%Y-%m-%d'))

## Temperature

In [19]:
temperature = [22, 22, 22, 22, 23, 21, 21, 21, 20, 20, 
               20, 20, 19, 19, 20, 19, 18, 18, 19, 19, 18]

## Time

In [20]:
feeding_time = [8, 7,8,8,9,9,9,8,8,9,12,9,10,8,8,9,9,10,9,10,10]

## Growth

In [21]:
initial_height = [2.2, 2.4, 2.7, 2.1, 2.8, 2.6, 2.9, 2.4, 2.1, 2.2,
                  2.4, 2.2, 2.9, 2.5, 2.6, 2.9, 2.4, 2.7, 2.0, 2.7, 2.6]

In [22]:
end_height = [7.2, 7.8, 7.9, 6.3, 6.1, 7.5, 6.6, 6.6, 6.1, 7.1,
              7.8, 6.6, 7.3, 7.9, 7.5, 6.5, 6.9, 6.1, 6.2, 7.0, 6.6]

## Bubbles

In [23]:
bubble_size = [0.2, 0.4, 0.3, 0.2, 0.1, 0.1, 0.4, 0.1, 0.3, 0.2, 0.4, 0.2, 0.1,
        0.3, 0.4, 0.2, 0.1, 0.1, 0.4, 0.4, 0.2]

## df

In [24]:
feedings_dict = {
    'feeding_date':feeding_dates, 
    'temperature':temperature,
    'feeding_time':feeding_time,
    'initial_height':initial_height,
    'end_height':end_height,
    'bubble_size':bubble_size
}

In [25]:
feedings = pd.DataFrame(data=feedings_dict)

In [26]:
feedings.to_parquet(PATH + 'feedings.parquet')

# Utilization

**TODO**
* create a suggestion table for suitable recipes
* store dfs and use new df for concat

## Baked breads

In [27]:
bread_name = ['Dinkelsonni', 'Italienisches Landbrot', 'Hausbrot', 'Neptune']

In [28]:
baking_date = ['2022-08-13', '2022-09-18', '2022-10-23', '2022-11-01']

In [29]:
bread_rating = [4, 5, 4, 5]

In [30]:
used_starter = [10, 20, 15, 10]

In [31]:
flour_type = [['spelt', 'wheat'], ['wheat'], ['wheat', 'rye'], ['wheat']]

In [32]:
baked_bread_dict = {
    'bread_name':bread_name,
    'baking_date':baking_date,
    'bread_rating':bread_rating,
    'used_starter':used_starter,
    'flour_type':flour_type
}

In [33]:
baked_bread = pd.DataFrame(data=baked_bread_dict)

In [34]:
baked_bread['baking_date'] = pd.to_datetime(baked_bread['baking_date'])

In [35]:
baked_bread.to_parquet(PATH + 'baked_bread.parquet')
baked_bread.to_csv(PATH + 'baked_bread.csv')

## Recycle utilizations

In [36]:
recycling_date = ['2022-09-13', '2022-10-18', '2022-11-23', '2022-12-01']

In [37]:
recyling = [True, True, True, True]

In [38]:
recyled_dough_dict = {
    'date':recycling_date,
    'recycling_happened':recyling
}

In [39]:
recyled_dough = pd.DataFrame(data=recyled_dough_dict)

In [40]:
recyled_dough.to_parquet(PATH + 'recycled_dough.parquet')

## Bread experiments TODO

**TODO**
* Vorgehen + Modifikation + Resultat + Experimentbasis (Brot) 

# Further data

## Bacteria composition

In [41]:
bacteria_composition = pd.DataFrame({
    'temperature':[20, 25,30, 35],
    'dominant_microbes':['none', 'lactic acid bacteria', 'Acetic acid bacteria', 'sourdough yeast']
})

bacteria_composition.to_parquet(PATH + 'bacteria_composition.parquet')

# Initial processing

In [42]:
feedings_processed = feedings.copy()

In [43]:
feedings_processed["bacteria_composition"] = np.where(
    feedings_processed["temperature"] <= 20,
    bacteria_composition.loc[
        bacteria_composition["temperature"] == 20, "dominant_microbes"
    ],
    np.where(
        ((feedings_processed["temperature"] > 20) & (feedings_processed["temperature"] <= 25)),
        bacteria_composition.loc[
            bacteria_composition["temperature"] == 25, "dominant_microbes"
        ],
        np.where(
            ((feedings_processed["temperature"] > 25) & (feedings_processed["temperature"] <= 30)),
            bacteria_composition.loc[
                bacteria_composition["temperature"] == 30, "dominant_microbes"
            ],
            bacteria_composition.loc[
                bacteria_composition["temperature"] == 35, "dominant_microbes"
            ],
        ),
    ),
)

In [44]:
# Growth rate
feedings_processed["growth_rate"] = (
    feedings_processed["end_height"] / feedings_processed["initial_height"]
)
feedings_processed["growth_rate_per_hour"] = (
    feedings_processed["end_height"]
    / feedings_processed["initial_height"]
    / feedings_processed["feeding_time"]
)

In [45]:
feedings_processed.to_parquet(PATH + 'feedings_processed.parquet')